In [126]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [218]:
train=pd.read_csv(r'D:\SarojOfficeWork\AI\Kaggle\Covid-19\covid19-local-us-ca-forecasting-week-1\ca_train.csv')
train.rename(columns={'Id': 'ForecastId'}, inplace=True)
test=pd.read_csv(r'D:\SarojOfficeWork\AI\Kaggle\Covid-19\covid19-local-us-ca-forecasting-week-1\ca_test.csv')

# Preprocessing of Train and test Data together
# Concatenate Train and Test Data with a new column to identify Train (Type=0) VS Test Data(Type=1)  
train['Type']=pd.DataFrame(np.zeros(len(train)).astype(int))
test['Type']=pd.DataFrame(np.ones(len(test)).astype(int))

print('Original Train Data shape:{} and Test Data shape:{}'.format(train.shape,test.shape))

features_Data=pd.concat([train.drop(columns=['ConfirmedCases','Fatalities']),test])
features_Data.drop(columns=['ForecastId','Province/State','Country/Region','Lat','Long'],inplace=True)
print('features_Data shape after dropping unnecesary columns ',features_Data.shape)

# Lets deal with Date columns
# First convert the  column to Date time data type
features_Data['Date']=pd.to_datetime(features_Data['Date'])

# We can also extract week, day, dayofweek,dayofyear and create separate columns
features_Data.insert(1,'Week',features_Data['Date'].dt.week)
features_Data.insert(2,'Day',features_Data['Date'].dt.day)
features_Data.insert(3,'DayofWeek',features_Data['Date'].dt.dayofweek)
features_Data.insert(4,'DayofYear',features_Data['Date'].dt.dayofyear)

# Check for Null Values
print('Null Value Check:\n',features_Data.isnull().sum())

train_Data=features_Data[features_Data.Type==0].drop(columns=['Date','Type'])
test_Data=features_Data[features_Data.Type==1].drop(columns=['Date','Type'])
del features_Data
print('Final Train Data shape:{} and Test Data shape:{}'.format(train_Data.shape,test_Data.shape))


Original Train Data shape:(63, 9) and Test Data shape:(43, 7)
features_Data shape after dropping unnecesary columns  (106, 2)
Null Value Check:
 Date         0
Week         0
Day          0
DayofWeek    0
DayofYear    0
Type         0
dtype: int64
Final Train Data shape:(63, 4) and Test Data shape:(43, 4)


In [228]:
train_Data.head()

Week  Day  DayofWeek  DayofYear
0     4   22          2         22
1     4   23          3         23
2     4   24          4         24
3     4   25          5         25
4     4   26          6         26

In [221]:
train.rename(columns={'ForecastId': 'ForecastID'}, inplace=True)
train

ForecastID Province/State Country/Region      Lat      Long        Date  \
0            1     California             US  36.1162 -119.6816  22-01-2020   
1            2     California             US  36.1162 -119.6816  23-01-2020   
2            3     California             US  36.1162 -119.6816  24-01-2020   
3            4     California             US  36.1162 -119.6816  25-01-2020   
4            5     California             US  36.1162 -119.6816  26-01-2020   
..         ...            ...            ...      ...       ...         ...   
58          59     California             US  36.1162 -119.6816  20-03-2020   
59          60     California             US  36.1162 -119.6816  21-03-2020   
60          61     California             US  36.1162 -119.6816  22-03-2020   
61          62     California             US  36.1162 -119.6816  23-03-2020   
62          63     California             US  36.1162 -119.6816  24-03-2020   

    ConfirmedCases  Fatalities  Type  
0                0           0     0  
1                0           0     0  
2                0           0     0  
3                0           0     0  
4                0           0     0  
..             ...         ...   ...  
58            1177          23     0  
59            1364          24     0  
60            1642          30     0  
61            2108          39     0  
62            2538          50     0  

[63 rows x 9 columns]

In [222]:
# Extract features and label
features=train_Data.values
label1=train.ConfirmedCases.values
label2=train.Fatalities.values

In [223]:
# Lets apply Stratified K-Fold Cross Validation
from sklearn.model_selection import StratifiedKFold
#from sklearn.metrics import f1_score,classification_report,confusion_matrix
from sklearn.metrics import mean_squared_error,median_absolute_error,r2_score

def stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y):
    global df_model_selection
    
    skf = StratifiedKFold(n_splits, random_state=12,shuffle=True)
    
    weighted_r2_score = []
    #print(skf.split(X,y))
    for train_index, test_index in skf.split(X,y):
        X_train, X_test = X[train_index], X[test_index] 
        y_train, y_test = y[train_index], y[test_index]
        
        
        model_obj.fit(X_train, y_train)##### HERE ###
        test_ds_predicted = model_obj.predict( X_test ) ##### HERE ####   
        #print( metrics.classification_report( y_test, test_ds_predicted ) )    
        weighted_r2_score.append(round(r2_score(y_true=y_test, y_pred=test_ds_predicted),2))
        
    sd_weighted_r2_score = np.std(weighted_r2_score, ddof=1)
    range_of_r2_scores = "{}-{}".format(min(weighted_r2_score),max(weighted_r2_score))    
    df_model_selection = pd.concat([df_model_selection,pd.DataFrame([[process,model_name,sorted(weighted_r2_score),range_of_r2_scores,sd_weighted_r2_score]], columns =COLUMN_NAMES) ])
    

In [182]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from xgboost import XGBRFRegressor
from sklearn.neighbors import KNeighborsRegressor

## Modeling for Predicting ConfirmedCases 

In [229]:
COLUMN_NAMES = ["Process","Model Name", "r2 Scores","Range of r2 Scores","Std Deviation of r2 Scores"]
df_model_selection = pd.DataFrame(columns=COLUMN_NAMES)

process='ConfirmedCases Prediction'
n_splits = 7
X=features
y=label1


# 1.LinearRegression
model_LR=LinearRegression()
model_obj=model_LR
model_name='LinearRegression'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # 2.BayesianRidge
model_BR=BayesianRidge()
model_obj=model_BR
model_name='BayesianRidgeRegressor'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # 3.DecisionTreeRegressor
model_DTR=DecisionTreeRegressor()
model_obj=model_DTR
model_name='DecisionTreeRegressor'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # 4.RandomForestRegressor
model_RFR=RandomForestRegressor()
model_obj=model_RFR
model_name='RandomForestRegressor'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # 5.GradientBoostingRegressor
model_GBR=GradientBoostingRegressor()
model_obj=model_GBR
model_name='GradientBoostingRegressor'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # # 6.XGBRegressor
model_XGBR=XGBRegressor()
model_obj=model_XGBR
model_name='XGBRegressor'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # # 7.XGBRFRegressor
model_XGBRFR=XGBRFRegressor()
model_obj=model_XGBRFR
model_name='XGBRFRegressor'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# 8.KNeighborsRegressor
model_KNNR=KNeighborsRegressor()
model_obj=model_KNNR
model_name='KNeighborsRegressor'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)


df_model_selection

Process                 Model Name  \
0  ConfirmedCases Prediction           LinearRegression   
0  ConfirmedCases Prediction     BayesianRidgeRegressor   
0  ConfirmedCases Prediction      DecisionTreeRegressor   
0  ConfirmedCases Prediction      RandomForestRegressor   
0  ConfirmedCases Prediction  GradientBoostingRegressor   
0  ConfirmedCases Prediction               XGBRegressor   
0  ConfirmedCases Prediction             XGBRFRegressor   
0  ConfirmedCases Prediction        KNeighborsRegressor   

                                         r2 Scores Range of r2 Scores  \
0   [-1.36, -0.21, -0.14, -0.04, -0.03, 0.0, 0.05]         -1.36-0.05   
0  [-0.56, -0.13, -0.07, -0.04, -0.03, -0.01, 0.0]          -0.56-0.0   
0        [0.88, 0.93, 0.95, 0.95, 0.95, 0.97, 1.0]           0.88-1.0   
0         [0.0, 0.82, 0.85, 0.9, 0.95, 0.95, 0.96]           0.0-0.96   
0         [0.0, 0.88, 0.92, 0.95, 0.95, 0.96, 1.0]            0.0-1.0   
0        [0.0, 0.86, 0.89, 0.92, 0.93, 0.95, 0.97]           0.0-0.97   
0         [0.0, 0.91, 0.92, 0.92, 0.97, 0.99, 1.0]            0.0-1.0   
0          [0.0, 0.57, 0.69, 0.76, 0.9, 0.91, 1.0]            0.0-1.0   

   Std Deviation of r2 Scores  
0                    0.498455  
0                    0.198830  
0                    0.036839  
0                    0.346259  
0                    0.358442  
0                    0.349639  
0                    0.361518  
0                    0.337639

In [188]:
## From Above Results Decesion Tree Regressor seems to be predicting better 
# Now lets try to get the Scores using StratifiedKFold Cross Validation

#Initialize the algo
model=DecisionTreeRegressor()

#Initialize StratifiedKFold Method
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=7, 
              random_state=1,
              shuffle=True)

#Initialize For Loop 

i=0
for train,test in kfold.split(features,label1):
    i = i+1
    X_train,X_test = features[train],features[test]
    y_train,y_test = label1[train],label1[test]
    
    model.fit(X_train,y_train)
    test_ds_predicted=model.predict(X_test)
    train_ds_predicted=model.predict(X_train)
    
    test_r2_score=round(r2_score(y_true=y_test, y_pred=test_ds_predicted ),2)
    train_r2_score=round(r2_score(y_true=y_train, y_pred=train_ds_predicted ),2)
    
    print("Train r2-Score: {}, Test r2-score: {}, for Sample Split: {}".format(train_r2_score,test_r2_score,i))

Train r2-Score: 1.0, Test r2-score: 0.97, for Sample Split: 1
Train r2-Score: 1.0, Test r2-score: 0.89, for Sample Split: 2
Train r2-Score: 1.0, Test r2-score: 0.95, for Sample Split: 3
Train r2-Score: 1.0, Test r2-score: 0.95, for Sample Split: 4
Train r2-Score: 1.0, Test r2-score: 1.0, for Sample Split: 5
Train r2-Score: 1.0, Test r2-score: 0.95, for Sample Split: 6
Train r2-Score: 1.0, Test r2-score: 0.95, for Sample Split: 7


In [232]:
#Lets extract the Train and Test sample for split 5
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=7, #n_splits should be equal to no of cv value in cross_val_score
              random_state=1,
              shuffle=True)
i=0
for train,test in kfold.split(features,label1):
    i = i+1
    if i == 5:
        X_train,X_test,y_train,y_test = features[train],features[test],label1[train],label1[test]


########################################################################################################
#Final Model
finalModel=DecisionTreeRegressor()
finalModel.fit(X_train,y_train)

test_ds_predicted=finalModel.predict(X_test)
train_ds_predicted=finalModel.predict(X_train)


test_r2_score=round(r2_score(y_true=y_test, y_pred=test_ds_predicted ),2)
train_r2_score=round(r2_score(y_true=y_train, y_pred=train_ds_predicted ),2)

print("Train r2-Score: {}, Test r2-score: {}".format(train_r2_score,test_r2_score))


train_score=np.round(finalModel.score(X_train,y_train),2)
test_score=np.round(finalModel.score(X_test,y_test),2)
print('Train Accuracy Score is:{} and  Test Accuracy Score:{}'.format(train_score,test_score))

Train r2-Score: 1.0, Test r2-score: 1.0
Train Accuracy Score is:1.0 and  Test Accuracy Score:1.0


## Prediction of ConfirmedCases

In [226]:
test=pd.read_csv(r'D:\SarojOfficeWork\AI\Kaggle\Covid-19\covid19-local-us-ca-forecasting-week-1\ca_test.csv')
df_submission=pd.DataFrame(finalModel.predict(test_Data),columns=['ConfirmedCases'])
df_submission.insert(0,'ForecastId',test.ForecastId)


In [227]:
df_submission

ForecastId  ConfirmedCases
0            1           221.0
1            2           282.0
2            3           340.0
3            4           426.0
4            5           557.0
5            6           698.0
6            7           751.0
7            8           952.0
8            9          1177.0
9           10          1364.0
10          11          1642.0
11          12          2108.0
12          13          2538.0
13          14          2538.0
14          15          2538.0
15          16          2538.0
16          17          2538.0
17          18          2538.0
18          19          2538.0
19          20          2538.0
20          21             0.0
21          22             0.0
22          23             0.0
23          24             0.0
24          25             0.0
25          26             0.0
26          27             0.0
27          28             0.0
28          29             0.0
29          30           144.0
30          31           177.0
31          32           221.0
32          33           282.0
33          34           340.0
34          35           340.0
35          36           751.0
36          37           751.0
37          38           751.0
38          39          1642.0
39          40          1642.0
40          41          1642.0
41          42          1642.0
42          43          2108.0

## Modeling for Predicting Fatalities 

In [196]:
COLUMN_NAMES = ["Process","Model Name", "r2 Scores","Range of r2 Scores","Std Deviation of r2 Scores"]
df_model_selection = pd.DataFrame(columns=COLUMN_NAMES)

process='Fatalities Prediction'
n_splits = 7
X=features
y=label2


# 1.LinearRegression
model_LR=LinearRegression()
model_obj=model_LR
model_name='LinearRegression'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # 2.BayesianRidge
model_BR=BayesianRidge()
model_obj=model_BR
model_name='BayesianRidgeRegressor'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # 3.DecisionTreeRegressor
model_DTR=DecisionTreeRegressor()
model_obj=model_DTR
model_name='DecisionTreeRegressor'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # 4.RandomForestRegressor
model_RFR=RandomForestRegressor()
model_obj=model_RFR
model_name='RandomForestRegressor'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # 5.GradientBoostingRegressor
model_GBR=GradientBoostingRegressor()
model_obj=model_GBR
model_name='GradientBoostingRegressor'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # # 6.XGBRegressor
model_XGBR=XGBRegressor()
model_obj=model_XGBR
model_name='XGBRegressor'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # # 7.XGBRFRegressor
model_XGBRFR=XGBRFRegressor()
model_obj=model_XGBRFR
model_name='XGBRFRegressor'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# 8.KNeighborsRegressor
model_KNNR=KNeighborsRegressor()
model_obj=model_KNNR
model_name='KNeighborsRegressor'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)


df_model_selection

Process                 Model Name  \
0  Fatalities Prediction           LinearRegression   
0  Fatalities Prediction     BayesianRidgeRegressor   
0  Fatalities Prediction      DecisionTreeRegressor   
0  Fatalities Prediction      RandomForestRegressor   
0  Fatalities Prediction  GradientBoostingRegressor   
0  Fatalities Prediction               XGBRegressor   
0  Fatalities Prediction             XGBRFRegressor   
0  Fatalities Prediction        KNeighborsRegressor   

                                           r2 Scores Range of r2 Scores  \
0    [-16.99, -0.89, -0.11, -0.08, -0.01, 0.0, 0.04]        -16.99-0.04   
0  [-8.18, -0.33, -0.05, -0.03, -0.03, -0.02, -0.01]        -8.18--0.01   
0         [0.81, 0.92, 0.93, 0.94, 0.94, 0.94, 0.94]          0.81-0.94   
0           [0.8, 0.81, 0.9, 0.92, 0.92, 0.94, 0.99]           0.8-0.99   
0         [0.76, 0.81, 0.94, 0.94, 0.94, 0.94, 0.98]          0.76-0.98   
0           [0.87, 0.88, 0.89, 0.9, 0.91, 0.98, 1.0]           0.87-1.0   
0            [0.68, 0.9, 0.9, 0.98, 0.98, 0.99, 1.0]           0.68-1.0   
0        [-0.74, 0.67, 0.82, 0.85, 0.87, 0.94, 0.97]         -0.74-0.97   

   Std Deviation of r2 Scores  
0                    6.363709  
0                    3.064234  
0                    0.047859  
0                    0.068972  
0                    0.082144  
0                    0.050803  
0                    0.113200  
0                    0.609941

In [197]:
## From Above Results XGBRegressor seems to be predicting better 
# Now lets try to get the Scores using StratifiedKFold Cross Validation

#Initialize the algo
model=XGBRegressor()

#Initialize StratifiedKFold Method
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=7, 
              random_state=1,
              shuffle=True)

#Initialize For Loop 

i=0
for train,test in kfold.split(features,label2):
    i = i+1
    X_train,X_test = features[train],features[test]
    y_train,y_test = label2[train],label2[test]
    
    model.fit(X_train,y_train)
    test_ds_predicted=model.predict(X_test)
    train_ds_predicted=model.predict(X_train)
    
    test_r2_score=round(r2_score(y_true=y_test, y_pred=test_ds_predicted ),2)
    train_r2_score=round(r2_score(y_true=y_train, y_pred=train_ds_predicted ),2)
    
    print("Train r2-Score: {}, Test r2-score: {}, for Sample Split: {}".format(train_r2_score,test_r2_score,i))

Train r2-Score: 1.0, Test r2-score: 1.0, for Sample Split: 1
Train r2-Score: 1.0, Test r2-score: 0.98, for Sample Split: 2
Train r2-Score: 1.0, Test r2-score: 0.17, for Sample Split: 3
Train r2-Score: 1.0, Test r2-score: 0.93, for Sample Split: 4
Train r2-Score: 1.0, Test r2-score: 0.91, for Sample Split: 5
Train r2-Score: 1.0, Test r2-score: 0.89, for Sample Split: 6
Train r2-Score: 1.0, Test r2-score: 0.93, for Sample Split: 7


In [198]:
#Lets extract the Train and Test sample for split 1
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=7, #n_splits should be equal to no of cv value in cross_val_score
              random_state=1,
              shuffle=True)
i=0
for train,test in kfold.split(features,label2):
    i = i+1
    if i == 1:
        X_train,X_test,y_train,y_test = features[train],features[test],label2[train],label2[test]


########################################################################################################
#Final Model
finalModel=DecisionTreeRegressor()
finalModel.fit(X_train,y_train)

test_ds_predicted=model.predict(X_test)
train_ds_predicted=model.predict(X_train)


test_r2_score=round(r2_score(y_true=y_test, y_pred=test_ds_predicted ),2)
train_r2_score=round(r2_score(y_true=y_train, y_pred=train_ds_predicted ),2)

print("Train r2-Score: {}, Test r2-score: {}".format(train_r2_score,test_r2_score))


train_score=np.round(finalModel.score(X_train,y_train),2)
test_score=np.round(finalModel.score(X_test,y_test),2)
print('Train Accuracy Score is:{} and  Test Accuracy Score:{}'.format(train_score,test_score))

Train r2-Score: 0.98, Test r2-score: 1.0
Train Accuracy Score is:1.0 and  Test Accuracy Score:0.97


## Prediction of Fatalities

In [208]:
df_submission.insert(2,'Fatalities',pd.DataFrame(finalModel.predict(test_Data),columns=['Fatalities']))
df_submission.astype('int').head()

ForecastId  ConfirmedCases  Fatalities
0           1             221           4
1           2             282           4
2           3             340           5
3           4             426           5
4           5             557           7

In [209]:
df_submission.astype('int').to_csv('submission1.csv',index=False)